<a href="https://colab.research.google.com/github/BhumikaAggwl/isro_proj/blob/main/isro_task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#extra tle tasksssssssssss
#for finding the texture of prithvi and showing the chanda mama


In [2]:
!pip install sgp4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.3/232.3 kB 8.5 MB/s eta 0:00:00


In [5]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from sgp4.api import Satrec, jday
from datetime import datetime, timedelta
import os
import matplotlib.image as mpimg
import pandas as pd

# Function to read the TLE file
def read_tle_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    tle_data = {}
    for i in range(0, len(lines), 3):
        satellite_name = lines[i].strip()
        tle1 = lines[i + 1].strip()
        tle2 = lines[i + 2].strip()
        tle_data[satellite_name] = [tle1, tle2]

    return tle_data

# Parse TLE data
def parse_tle(satellite_name, tle_data):
    line1, line2 = tle_data[satellite_name]
    satrec = Satrec.twoline2rv(line1, line2)
    return satrec

# Predict satellite's position over time
def predict_position(satrec, start_time, end_time, step_minutes=10):
    positions = []
    times = []

    current_time = start_time
    while current_time <= end_time:
        jd, fr = jday(current_time.year, current_time.month, current_time.day,
                      current_time.hour, current_time.minute, current_time.second)
        e, r, v = satrec.sgp4(jd, fr)

        if e == 0:  # Successful propagation
            x, y, z = r
            positions.append([x, y, z])
            times.append(current_time)

        current_time += timedelta(minutes=step_minutes)

    return times, positions

# Function to normalize image data to [0, 1] range
def normalize_image(image):
    return image / 255.0

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.image as mpimg
import os

# Function to normalize image data to [0, 1] range
def normalize_image(image):
    return image / 255.0

# Visualize the orbit with the Earth and Moon in 3D with textures
def visualize_orbit_with_earth_and_moon(times, positions, satellite_name, earth_texture_path, moon_texture_path, output_folder="output_images"):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Load the Earth and Moon texture images
    earth_texture = mpimg.imread(earth_texture_path)  # Earth texture image
    moon_texture = mpimg.imread(moon_texture_path)  # Moon texture image

    # Normalize the images to [0, 1] range
    earth_texture = normalize_image(earth_texture)
    moon_texture = normalize_image(moon_texture)

    # Define Earth and Moon radii
    earth_radius = 6371  # Earth's radius in kilometers
    moon_radius = 1737  # Moon's radius in kilometers

    # Generate spherical coordinates for Earth and Moon
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)
    x_earth = earth_radius * np.outer(np.cos(u), np.sin(v))
    y_earth = earth_radius * np.outer(np.sin(u), np.sin(v))
    z_earth = earth_radius * np.outer(np.ones(np.size(u)), np.cos(v))

    x_moon = moon_radius * np.outer(np.cos(u), np.sin(v))
    y_moon = moon_radius * np.outer(np.sin(u), np.sin(v))
    z_moon = moon_radius * np.outer(np.ones(np.size(u)), np.cos(v))

    # Create the figure for 3D plotting
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Plot Earth with texture
    ax.plot_surface(x_earth, y_earth, z_earth, rstride=1, cstride=1, facecolors=earth_texture, alpha=0.7, shade=True)
    ax.set_title(f"Earth with Texture")
    ax.set_box_aspect([1, 1, 1])  # Equal aspect ratio

    # Save Earth texture visualization
    earth_img_filename = os.path.join(output_folder, f"{satellite_name}_earth_texture.png")
    plt.savefig(earth_img_filename, dpi=300)
    plt.cla()

    # Plot Moon with texture
    ax.plot_surface(x_moon, y_moon, z_moon, rstride=1, cstride=1, facecolors=moon_texture, alpha=0.6, shade=True)
    ax.set_title(f"Moon with Texture")
    ax.set_box_aspect([1, 1, 1])  # Equal aspect ratio

    # Save Moon texture visualization
    moon_img_filename = os.path.join(output_folder, f"{satellite_name}_moon_texture.png")
    plt.savefig(moon_img_filename, dpi=300)
    plt.cla()

    # Plot Satellite's Orbit with Earth and Moon in the same figure
    ax.plot_surface(x_earth, y_earth, z_earth, rstride=1, cstride=1, facecolors=earth_texture, alpha=0.7, shade=True)
    ax.plot_surface(x_moon, y_moon, z_moon, rstride=1, cstride=1, facecolors=moon_texture, alpha=0.6, shade=True)

    x_vals = [pos[0] for pos in positions]
    y_vals = [pos[1] for pos in positions]
    z_vals = [pos[2] for pos in positions]
    ax.plot(x_vals, y_vals, z_vals, label="Orbit", color='r')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f"Satellite Orbit with Earth and Moon: {satellite_name}")
    ax.set_box_aspect([1, 1, 1])  # Equal aspect ratio

    # Save the full visualization
    orbit_img_filename = os.path.join(output_folder, f"{satellite_name}_orbit_with_earth_and_moon.png")
    plt.savefig(orbit_img_filename, dpi=300)
    plt.close()

    return earth_img_filename, moon_img_filename, orbit_img_filename


# Save positions to CSV
def save_to_csv(times, positions, satellite_name, filename="satellite_orbit.csv"):
    df = pd.DataFrame(positions, columns=["X", "Y", "Z"])
    df['Time'] = times
    df = df[['Time', 'X', 'Y', 'Z']]
    df.to_csv(filename, index=False)

# Main Function
def main():
    # Load TLE Data from Text File
    tle_data = read_tle_file("last30.txt")  # Replace with your actual TLE data file

    # List all available satellites
    print("Available Satellites:")
    for i, satellite_name in enumerate(tle_data.keys(), 1):
        print(f"{i}. {satellite_name}")

    # Get user input to select a satellite
    satellite_choice = int(input(f"Select a satellite (1-{len(tle_data)}): "))
    satellite_name = list(tle_data.keys())[satellite_choice - 1]

    # Parse TLE Data for the chosen satellite
    satrec = parse_tle(satellite_name, tle_data)

    # Define time range (e.g., next 24 hours)
    start_time = datetime.utcnow()
    end_time = start_time + timedelta(hours=24)

    # Predict satellite position over time
    times, positions = predict_position(satrec, start_time, end_time)

    # Save predicted positions to CSV
    save_to_csv(times, positions, satellite_name)

    # Define Earth and Moon texture paths
    earth_texture_path = 'earth_texture.jpg'  # Replace with the actual path to your Earth texture image
    moon_texture_path = 'moon_texture.jpg'    # Replace with the actual path to your Moon texture image

    # Visualize the orbit with Earth texture and Moon
    visualize_orbit_with_earth_and_moon(times, positions, satellite_name, earth_texture_path, moon_texture_path)

if __name__ == "__main__":
    main()



Available Satellites:
1. STARLINK-11465 [DTC]
2. STARLINK-11426 [DTC]
3. STARLINK-11454 [DTC]
4. STARLINK-11473 [DTC]
5. STARLINK-11429 [DTC]
6. STARLINK-11470 [DTC]
7. STARLINK-11427 [DTC]
8. STARLINK-11456 [DTC]
9. STARLINK-11474 [DTC]
10. STARLINK-11463 [DTC]
11. STARLINK-11472 [DTC]
12. STARLINK-11467 [DTC]
13. STARLINK-11460 [DTC]
14. STARLINK-32604
15. STARLINK-32553
16. STARLINK-32603
17. STARLINK-32444
18. STARLINK-32619
19. STARLINK-32580
20. STARLINK-32617
21. SZ-18 MODULE
22. CZ-2C R/B
23. SUPERVIEW NEO-2 03
24. SUPERVIEW NEO-2 04
25. KINEIS-5C
26. KINEIS-5D
27. KINEIS-5E
28. KINEIS-5A
29. KINEIS-5B
30. ELECTRON KICK STAGE R/B
31. STARLINK-11384 [DTC]
32. STARLINK-11379 [DTC]
33. STARLINK-11387 [DTC]
34. STARLINK-11389 [DTC]
35. STARLINK-11388 [DTC]
36. STARLINK-11345 [DTC]
37. STARLINK-11381 [DTC]
38. STARLINK-11369 [DTC]
39. STARLINK-11303 [DTC]
40. STARLINK-11367 [DTC]
41. STARLINK-11386 [DTC]
42. STARLINK-11380 [DTC]
43. STARLINK-33566
44. STARLINK-33545
45. STARLINK-335